In [85]:
from tfmc.loader import load_tf_model
from tfmc.schema_gen import generate_schema

In [86]:
schema = generate_schema()
model = load_tf_model("aws_ec2_ebs_docker_host")

resources = model["resources"]

Objective: Generate

In [87]:
elements = []
associations = []
attributes = []

def handle_assoc_prop(source_catid, mm_id, target_cat, target_id, mm_category):
    
    # If assert fails: manually handle this exception through custom.yaml
    assert target_cat == mm_category
    assoc = source_catid, mm_id, f"{target_cat}::{target_id}"
    return assoc

for res_catid, res in resources.items():
    elem, id = res_catid.split(".")

    source_catid = f"{elem}::{id}"

    for pk, prop in res.items():
        if pk != '__tfmeta':
            mm_id = f"{elem}::{pk}"
            if mm_assoc := schema.associations.get(mm_id):
                # print(f"{mm_id} -> assoc")
                # print(mm_assoc)
                # print(prop)
                try:
                    if mm_assoc.get('mm_nested_block'):
                        print("TODO: Handle nested blocks")
                    else:
                        mm_cat = mm_assoc.get('mm_type')
                        if targets := prop.get('__attributes__'): # more than 1 target
                            for target in targets:
                                ts = target.split(".")
                                tcat = ts[-3]
                                tid = ts[-2]
                                associations.append(
                                    handle_assoc_prop(source_catid, mm_id, tcat, tid, mm_cat))
                        elif target := prop.get('__attribute__'): # just 1 target
                            target_cat =  prop.get('__type__')
                            target_id = prop.get('__name__')
                            associations.append(
                                handle_assoc_prop(source_catid, mm_id, target_cat, target_id, mm_cat))
                except AttributeError:
                    print(f"TODO: Handle assoc {type(prop)} ids")
            elif mm_attr := schema.attributes.get(mm_id):
                # print(f"{mm_id} -> attrs")
                # TODO: Handle weird types from the MM
                value_and_type = prop, mm_attr.get('type')

                attr = source_catid, mm_id, value_and_type
                attributes.append(attr)
            else:
                print(f"Skipping {mm_id}: not in MM")

    
    # check if resource is supported:
    if elem in schema.categories:
        elements.append(elem)
    else:
        print(f"unsupported category: {elem}")

TODO: Handle assoc <class 'str'> ids
aws_instance::availability_zone -> attrs
('aws_instance::this', 'aws_instance::availability_zone', (None, 'string'))
Skipping aws_instance::connection: not in MM
aws_instance::id -> attrs
('aws_instance::this', 'aws_instance::id', ('1afb7d50-edbf-45f9-81c5-982886b3a370', 'string'))
aws_instance::instance_type -> attrs
('aws_instance::this', 'aws_instance::instance_type', ('t2.micro', 'string'))
('aws_instance::this', 'aws_instance::key_name', 'aws_key_pair::this')
Skipping aws_instance::provisioner: not in MM
TODO: Handle nested blocks
('aws_instance::this', 'aws_instance::subnet_id', 'aws_subnet::this')
aws_instance::tags -> attrs
('aws_instance::this', 'aws_instance::tags', (None, ['map', 'string']))
aws_instance::user_data -> attrs
('aws_instance::this', 'aws_instance::user_data', ('c76c92caab5680f9623e7da1edf121efe95546f6', 'string'))
aws_instance::volume_tags -> attrs
('aws_instance::this', 'aws_instance::volume_tags', (None, ['map', 'string'])

In [88]:
elements

['aws_instance',
 'aws_key_pair',
 'aws_security_group',
 'aws_security_group',
 'aws_security_group_rule',
 'aws_security_group_rule']

In [89]:
associations

[('aws_instance::this', 'aws_instance::key_name', 'aws_key_pair::this'),
 ('aws_instance::this', 'aws_instance::subnet_id', 'aws_subnet::this'),
 ('aws_instance::this',
  'aws_instance::vpc_security_group_ids',
  'aws_security_group::this'),
 ('aws_instance::this',
  'aws_instance::vpc_security_group_ids',
  'aws_security_group::that'),
 ('aws_security_group::that', 'aws_security_group::vpc_id', 'aws_vpc::this'),
 ('aws_security_group::this', 'aws_security_group::vpc_id', 'aws_vpc::this'),
 ('aws_security_group_rule::incoming_ssh',
  'aws_security_group_rule::security_group_id',
  'aws_security_group::this'),
 ('aws_security_group_rule::outgoing_any',
  'aws_security_group_rule::security_group_id',
  'aws_security_group::this')]

In [90]:
attributes

[('aws_instance::this', 'aws_instance::availability_zone', (None, 'string')),
 ('aws_instance::this',
  'aws_instance::id',
  ('1afb7d50-edbf-45f9-81c5-982886b3a370', 'string')),
 ('aws_instance::this', 'aws_instance::instance_type', ('t2.micro', 'string')),
 ('aws_instance::this', 'aws_instance::tags', (None, ['map', 'string'])),
 ('aws_instance::this',
  'aws_instance::user_data',
  ('c76c92caab5680f9623e7da1edf121efe95546f6', 'string')),
 ('aws_instance::this',
  'aws_instance::volume_tags',
  (None, ['map', 'string'])),
 ('aws_key_pair::this',
  'aws_key_pair::id',
  ('e6c48d54-e974-4d46-84f9-9092bbc603ca', 'string')),
 ('aws_key_pair::this', 'aws_key_pair::public_key', (None, 'string')),
 ('aws_security_group::that',
  'aws_security_group::id',
  ('3bdb5360-7920-4516-860c-7aa924b2b45a', 'string')),
 ('aws_security_group::that',
  'aws_security_group::tags',
  (None, ['map', 'string'])),
 ('aws_security_group::this',
  'aws_security_group::id',
  ('d77d9ff3-a5c8-42b7-8804-8e3fa5f17